

# load data

In [1]:
import time
import numpy as np
import pandas as pd

import sys
sys.path.append('/home/ngrav/project/')
from wearables.scripts import data as weardata
from wearables.scripts import eval_ as weareval

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
import fastdtw
import lightgbm as lgb
from sktime.datatypes._panel._convert import from_2d_array_to_nested
from sktime.regression.compose._ensemble import ComposableTimeSeriesForestRegressor
from sktime.classification.compose import ComposableTimeSeriesForestClassifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier


In [41]:
# data = weardata.dataloader(target_name='GA')

# models dev

In [2]:
class kNNTS():
    '''
    Arguments:
      data (weardata class): dataloader class from data.py
    '''
    def __init__(self, n_trials=10, target_name='GA'):
        self.n_trials = n_trials
        self.target_name = target_name
    
        
    def load_data(self):
        return weardata.dataloader(target_name=self.target_name)
        
    def fit(self, data):
        if data.kfold > 1:
            cv_eval = {}
            for k, cv_fold in enumerate(data.Xy_train.keys()):
                print('    cv_fold: ', cv_fold)
                [(X_train, y_train), (X_val, y_val)] = data.Xy_train[cv_fold]
                X_train, X_val = from_2d_array_to_nested(X_train), from_2d_array_to_nested(X_val)
                knn = KNeighborsTimeSeriesClassifier(n_neighbors=5, distance="dtw", n_jobs=-1)
                knn.fit(X_train, y_train)
                eval_metrics = weareval.eval_output(knn.predict(X_val), y_val, tasktype=data.tasktype)
                cv_eval[cv_fold] = {'model': knn, 
                                    # 'data': [(X_train, y_train), (X_val, y_val)], # store just IDs?
                                    'metric': eval_metrics['mae'] if data.tasktype=='regression' else eval_metrics['balanced_acc_adj'],
                                    'metrics': eval_metrics}
            # retain only best model
            tmp = {cv_fold:cv_eval[cv_fold]['metric'] for cv_fold in cv_eval.keys()}
            bst_fold = min(tmp, key=tmp.get) if data.tasktype=='regression' else max(tmp, key=tmp.get)
            self.knn = cv_eval[bst_fold]['model']
            return {'model': self.knn, 'metrics': cv_eval[bst_fold]['metrics']}
        else:
            X_train, y_train = data.Xy_train
            X_val, y_val = data.Xy_val
            X_train, X_val = from_2d_array_to_nested(X_train), from_2d_array_to_nested(X_val)
            self.knn = knn = KNeighborsTimeSeriesClassifier(n_neighbors=5, distance="dtw", n_jobs=-1)
            self.knn.fit(X_train, y_train)
            eval_metrics = weareval.eval_output(self.knn.predict(X_val), y_val, tasktype=data.tasktype)
            return {'model': self.knn, 'metrics': eval_metrics}
    
    def eval_test(self, data):
        X_test, y_test = data.Xy_test
        X_test = from_2d_array_to_nested(X_test)
        eval_metrics = weareval.eval_output(self.knn.predict(X_test), y_test, tasktype=data.tasktype)
        return eval_metrics
    
    def del_model(self):
        del self.knn
    
    def run_trials(self, verbose=True):
        if verbose:
            t_start = time.time()
            print('Starting kNTimeSeries trials; predict {}'.format(self.target_name))
        results = {}
        for n in range(self.n_trials):
            if verbose:
                tic = time.time()
            data = self.load_data()
            self.fit(data)
            results[n] = self.eval_test(data)
            self.del_model()
            if verbose:
                print('  finished trial {} in {:.2f}-s\t{:.1f}-min elapsed'.format(n, time.time()-tic, (time.time()-t_start)/60))
        return results

In [ ]:
results = kNNTS(n_trials=2).run_trials()
results 

Starting kNTimeSeries trials; predict GA
    cv_fold:  0


/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1403: UserWarning: Cython DTW is deprecated from V0.10
  out[i, j] = metric(X[i], Y[j], **kwds)
IOStream.flush timed out
IOStream.flush timed out


In [8]:
results = pd.DataFrame(results).T
results['model'] = 'RandomForest'

In [9]:
results

,mae,mape,rho,P_rho,model
0,8.306864,0.579775,0.254155,1.120509e-08,RandomForest
1,8.332274,0.607974,0.221692,9.317345e-07,RandomForest
